## **Machine Learning for Social Media-Based Depression Analysis** 

## Business Understanding

Mental health is an urgent issue globally, with depression affecting millions of individuals across all demographics. 
The internet, and particularly social media, has become a place where people often express their struggles, including depression. 
Early detection of depression symptons can be crucial in providing timely support or intervention. This project’s goal is to leverage data science to better understand and detect depressive expressions in online platforms, potentially paving the way for more proactive mental health support. Given the widespread use of platforms like Reddit, this research could benefit individuals by increasing awareness and intervention opportunities

***

## Data Understanding

Datasource: https://www.kaggle.com/datasets/rishabhkausish/reddit-depression-dataset/data 

The dataset already includes several key features that can be used to analyze and predict depression indicators based on Reddit posts. Specifically, the data has 7 key columns:


**subreddit** : The subreddit where each post was made, with posts from *"Depression"* and *"SuicideWatch"* labeled as 1 for depression and posts from other subreddits labeled as 0 (non-depression).

**title**: The title of the Reddit post.

**body**: The full text of the Reddit post, which may contain valuable information for understanding the context, tone, and possible indicators of depression.

**upvotes**: Number of upvotes each post received, which may indicate the post's visibility or resonance with the community.

**created_utc**: The timestamp of when the post was created in UTC, which can help in analyzing temporal trends.

**num_comments**: The number of comments on each post, which could provide insights into community engagement.

**label**: The target variable indicating depression (1) or non-depression (0) based on subreddit.
....
The raw data was collected  from five Reddit subreddits (sub topics), categorized based on their content. These included: Teenagers, Depression, SuicideWatch, DeepThoughts, Happy

Since the data is already collected from Reddit, with over 6 million rows, further data acquisition may not be necessary. Infact, the team proposes reducing the dataset to about 500,000 rows for the purpose of this project(and to save our laptops). However, if additional data is needed, we could plan to scrape Reddit for more recent posts using a tool that we could identify through further research, provided we comply with Reddit's data collection policies and privacy standards.


****

## Data Preparation

The dataset was reduced since it was too huge to work with.The data was reduced to 98826 features with 6 columns.there are null values in body and num_comments.

Import the necessary libraries

In [41]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt', quiet=True)
np.random.seed(0)

nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...


True

In [42]:
df=pd.read_csv(r"C:\Users\user\Documents\final_project\Data\reduced_reddit.csv")
df

,subreddit,title,body,upvotes,created_utc,num_comments,label
0,teenagers,Day 18 of doing 50 push-ups,NaN,4.0,1.619357e+09,4.0,0.0
1,teenagers,There isn’t a better feeling than finishing yo...,Then your teacher hits you with that “ Good jo...,7.0,1.554103e+09,NaN,0.0
2,teenagers,LMAOOO I can only get this guy to talk to me i...,Yeahhh maybe not babe,4.0,1.649342e+09,12.0,0.0
3,teenagers,This isn't going to work out,NaN,236.0,1.417630e+09,33.0,0.0
4,teenagers,Am I the only guy who found it hurtful as a ch...,\n\nLike... why? How is that funny? How does ...,6.0,1.569280e+09,4.0,0.0
...,...,...,...,...,...,...,...
98821,depression,Has anyone ever had any problems with gangs?,I don't care if my grammer isn't good so dont ...,8.0,1.558082e+09,1.0,1.0
98822,SuicideWatch,"Moments away from killing myself, not even bot...","I don't want to talk about my problems, I don'...",12.0,1.350754e+09,8.0,1.0
98823,SuicideWatch,I've grown used to being like this for years.,"I'm not suicidal at the moment, in fact I feel...",14.0,1.432074e+09,4.0,1.0
98824,depression,This speech from The Lord of the Rings really ...,"""It's like in the great stories, Mr. Frodo. Th...",323.0,1.355673e+09,28.0,1.0


In [43]:
#there are a total of 98826 features,null values are in body and num_comment
#we will drop null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98826 entries, 0 to 98825
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subreddit     98826 non-null  object 
 1   title         98826 non-null  object 
 2   body          80371 non-null  object 
 3   upvotes       98826 non-null  float64
 4   created_utc   98826 non-null  float64
 5   num_comments  94297 non-null  float64
 6   label         98826 non-null  float64
dtypes: float64(4), object(3)
memory usage: 5.3+ MB


In [44]:
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77051 entries, 2 to 98825
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subreddit     77051 non-null  object 
 1   title         77051 non-null  object 
 2   body          77051 non-null  object 
 3   upvotes       77051 non-null  float64
 4   created_utc   77051 non-null  float64
 5   num_comments  77051 non-null  float64
 6   label         77051 non-null  float64
dtypes: float64(4), object(3)
memory usage: 4.7+ MB


### Text cleaning and Preprocessing

In [45]:
#lower casing the text
df['title'] = df['title'].str.lower().copy()
low_column=df[['subreddit','body']]
for col in low_column:
    df[col]=df[col].str.lower()



C:\Users\user\AppData\Local\Temp\ipykernel_16620\3471966554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].str.lower().copy()
C:\Users\user\AppData\Local\Temp\ipykernel_16620\3471966554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col]=df[col].str.lower()


In [58]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import unicodedata

In [52]:
#removing special characters
def preprocessing_text(text):
    #remove punctuations
    text = text.translate(str.maketrans('','', string.punctuation))
    #remove numbers
    text= re.sub(r'd/+', '',text)
    #remove stopwords
    stop_word=set(stopwords.words('english'))
    text=' '.join([word for word in text.split() if word not in stop_word])
    #tokenization
    tokens=word_tokenize(text)
    #lemmatization
    lemitizer= WordNetLemmatizer()
    tokens= [lemitizer.lemmatize(word) for word in tokens]
    #rejoin token
    text=' '.join(tokens)
    #remove extra whitespace
    text=' '.join(text.split())
    return text
    

In [ ]:
#preprocess tittle column
df['title'] = df['title'].apply(preprocessing_text)

C:\Users\user\AppData\Local\Temp\ipykernel_16620\548776836.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].apply(preprocessing_text)


In [ ]:
#preprocess sureddit column
df['subreddit'] = df['subreddit'].apply(preprocessing_text)

C:\Users\user\AppData\Local\Temp\ipykernel_16620\33541029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subreddit'] = df['subreddit'].apply(preprocessing_text)


In [55]:
#preprocess body column
df['body'] = df['body'].apply(preprocessing_text)

C:\Users\user\AppData\Local\Temp\ipykernel_16620\3572614767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['body'] = df['body'].apply(preprocessing_text)


In [59]:
#removing emojis from the title column

def remove_emojis(text):
    # This pattern matches a wide range of emojis, including standard and extended ones
    emoji_pattern = re.compile(
        "[" 
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U000024C2-\U0001F251" 
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

remove_emojis(df['title'])


TypeError: expected string or bytes-like object